<img src="images/azure-snow.PNG" width="500" align="left">

<b>By: Taylor Ramsey</b>

### Existing Problem - What is the goal of this project
Currently, ServiceNow keeps track of metrics for the Support and EUC teams, and while the platform is robust, it is lacking a few key features. One of those is the ability to track the assigned incident backlog in order to measure to how the team is handling their task load. In order to chart backlog growth/decline, a metric has to be captured and recorded daily.

The backlogs being graphed here track incomplete incidents and tasks that are assigned to Support and EUC staff. The metric for each graph is recorded daily at 7pm by a python-requests ServiceNow API call. The backglod_data is appended to a small dataframe with a timestamp, and that information is written out to an Azure blob container for record-keeping, then later read into an Azure web application for charting historic data.

### This project consists of 5 Azure services:

* Azure VM - runs a daily API call to ServiceNow
* Azure App Service -  runs an instance of a flask web app which serves as the dashboard
* Azure Container Registry -  hands a docker image to the app service for deployment
* Azure KeyVault - stores ServiceNow login data used for the API call
* Azure Blob Storage - stores .csv files for record-keeping

### Flow of data within the application
![title](images/ServiceNow-WebApp.png)

# Overview of services used

### Remote Repository (Github/Azure Repos):

<img src="images/azure_repos.PNG" align="right" width="200px"/>
The initial code was developed and tested on my local ubuntu machine (using VSCode and Jupyter) and pushed to <br>both Github and Azure Repos. <br><br>For this project, I am including links to my Azure Repo for both the API call, and the Dash app:
<br clear="left"/>

[Dash App Repo](https://dev.azure.com/pythonprojects4cats/_git/ServiceNow-Dash-App)

[API Call Repo](https://dev.azure.com/pythonprojects4cats/ServiceNow-API-Call/_git/ServiceNow-API-Call)

### Azure App Service:
The Azure App Service deploys a small dash application within a docker container running an ubuntu base image. The docker image is created with the application files already included, so there is no need to pull code from a remote repo (for security purposes). Those files consist of:
* requirements.txt - for use with pip to install dependencies
* application.py - which contains the code for the dash page (based on the flask web framework)
* flask_secrets.py - which contains sensitive variables (added to .gitignore so it will not be shared in the repo) 

I'm using dash because it integrates well with plotly graphs, which I think provide a useful layer of interactivity for the viewer (you can hover and zoom/select). The app service is configured to reboot daily, shortly after the API call, in order to refresh the content being dislayed (as the service loads data from the blob each time it starts).

### Azure Virtual Machine:

The virtual machine contains python code which is run @ 7pm daily using a cron scheduler. The code:
* Reads an existing blob .csv from azure storage (if one does not exist, it will be created)
* Runs the ServiceNow api call to gather daily backlog data (using credentials which are imported from Azure Key Vault)
* Appends the backlog_daily_count information to a dataframe (and includes a timestamp of time run)
* Then, appends the dataframe to the existing blob container (overwriting the existing csv)

This code is also written exclusively in python, using imported Azure libraries for blob and keyvault functionality. The pandas library is also heavily used for data manipulation, and dataframe appending. In order to access the Azure Key Vault, this VM has also been given a managed identity.

### Azure Container Registry:
Azure container registry is a secure storage location for docker images, and a great alternative to Dockerhub if already using Azure for application deployments. The Azure container registry here holds the "snow-backlog" docker image, and passes it off to the app service for deployment online.

<img src="images/docker_code.PNG" width="300" align="left">
<img src="images/Docker-Logo-White-RGB_Horizontal.png" width="370" align="center" >

The docker image shown here uses an Ubuntu base image (v18.04). 
* The Dockerfile first installs python3 and pip installer
* An application directory ("app") is created in the home directory
* Files from the directory in which Docker is being run are added to the newly created app directory
* Additonal dependencies are installed (as defined in the requirements.txt file)
* "python3" is now declared as the entrypoint (which tells the container to use python to run my code)
* Finally, Docker runs the application.py file which loads the Flask page (it is included in the application file that docker run in port 80, which is the default expectation within Azure

### Azure Blob Storage:
Azure Blob storage is Microsoft's object storage solution for the cloud. I initially chose Azure SQL DB as a method of storage for the historical data, but due to the high cost, I have switched to Azure Blob Containers. When the daily API call is run, the contents of the Azure blobs are read, a new row of data is appended, and the existing blob is replaced/overwritten. Connection to the Azure blob is granted using a connection string hidden in the secrets file.

### Azure Key Vault:

Azure Key Vault is a cloud service that provides a secure storage environment for keys, passwords, certificates, and other secrets.
After configuring the Azure VM with a managed identity, I gave that identity access to the key vault, just like you would give to any other security principal (giving access to this VM only). The secrets being referenced here are my login credentials for ServiceNow-Prod. If I need to change the password for ServiceNow, the change can simply be updated in Azure Key Vault without the need for changing my application code (fewer deployments)

### Learning Experience:

I recently earned my AZ-104, and am always working to improve my Python abilities, so the opportunity to deploy a cloud-based application from scratch was very exciting. This was a perfect opportunity to learn how some of the core Azure/Cloud services work together for providing reliability, scalability, and automation.

I also got to brush up on my Ubuntu skills, which was by design, though that portion of the project should be revised in the future. In order to make this a completely serverless application, I'll need to deploy my code to Azure Functions. This change, along with Ansible automated deployment, will be coming soon. Having to move code from the dev environment into a new docker file, then testing the docker image before deploying to Azure made me very aware of the benefits of CI/CD. As someone who is interested in pursuing automation/DevOps as a career, I'm very much looking forward to accomplishing this goal in the coming weeks.

### Issues faced:

And of course I faced a ton of issues. A few are highlighted below:

* Cloud authentication is flexible, and offers many paths to success, but managed identity was new to me and took some time to understand
* Time issues with the ServiceNow data (the data from ServiceNow was UTC-standard, and needed to be converted to local time)
* Time issues with the Ubuntu machine (a data review/audit led to the discovery that the Ubuntu VM was in a different timezone - affecting the timing of cron jobs, resulting in poor data)
* Secret-keeping in the application file within the VM (cronjob would not detect my existing environmental variables, so I moved sensitive variables to an external file)
* Docker base image issues wherein the Alpine versions did not mingle with core data science libraries/dependencies (pandas, numpy)

### Final Product:

As of now, the completed project has been running for 3 days (with multiple testing instances before-hand). It meets the needs of the EUC and Support teams by giving us visibility and insight into the volume of our workloads over time. Below the link to "Learn about this site", I've also included a link to a ServiceNow exploratory data analysis (using python data analysis libraries).

<img src="images/final_product.PNG">

Thanks for viewing!

[LinkedIn Profile for Taylor Ramsey](https://www.linkedin.com/in/william-ramsey-777241119/)